# Extract

In [1]:
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [3]:
data = r"C:\Users\bputra\OneDrive - ESRI Indonesia\PT ESRI Indonesia\Research\code-script\17_scrapping_gmaps\gmaps\input\kabkot_list_and_xy_sample.csv"
df = pd.read_csv(data, delimiter=";")
pd.DataFrame(df)

,kakot,Shape_Leng,Shape_Area,lat,long
0,Kota Administrasi Jakarta Selatan,0.848568,0.011820,-6.272567,106.810043
1,Kota Bogor,0.708177,0.009106,-6.593494,106.799357
2,Kota Padang,1.881366,0.056664,-0.899532,100.437912
3,Magelang,2.016873,0.093674,-7.502515,110.249411
4,Pacitan,2.715767,0.117653,-8.124997,111.178150


In [4]:
data_lat = df['lat']
data_long = df['long']

> All Function 

#### Arsip Version 1.0

In [20]:

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except:
        return False
    return True


def scroll_to_bottom(iteration=None):
    xpath_sidebar = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    if(iteration == None):
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
        time.sleep(2)
    else:
        for i in range(1, iteration):
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
            time.sleep(2)


def scrape_data():
    # Daftar XPath untuk setiap informasi
    xpath_urls = [
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[1]/div[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[3]/div/span[2]/span/span[1]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[2]/span[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[2]/span[2]/span[2]'
    ]
    xpath_href = ['/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a']
    #xpath_href = ['//a[@class="hfpxzc"]']
    
    # List untuk menyimpan semua data
    all_data = []

    # Loop untuk mengekstrak informasi dari setiap elemen
    for i in range(3, 12, 2):  # misalnya, untuk nomor 3, 5, 7, 9, 11
        data = []
        for xpath in xpath_urls:
            xpath = xpath.format(i)
            #print(xpath)
            try :
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, xpath)))
                element = driver.find_element(By.XPATH, xpath)
                information = element.text
                data.append(information)
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                data.append('')
                
        for xpath_xy in xpath_href:
            xpath_xy = xpath_xy.format(i)
            try:
                elements = driver.find_elements(By.XPATH, xpath_xy)
                for element_xy in elements:
                    information_href = element_xy.get_attribute('href')
                    #information_data = element.get_attribute('aria-label')
                    data.append(information_href)
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                data.append(('',''))
        
        all_data.append(data)
    try :    
        # Buat DataFrame dari list all_data
        df = pd.DataFrame(all_data, columns=['poi_name', 'ratings', 'tags', 'address', 'telp', 'url'])
        # Simpan DataFrame ke dalam file CSV
        df.to_csv('scraped_data_testing4.csv', sep=';', index=False)
    except Exception as e :
        print(f"Error bagian saving data to csv {e}")
        
    # Cetak semua data yang telah diambil
    for info in all_data:
        print(info)
            
def url_hit(data_latitude, data_longitude) :
    try : 
        for latitude, longitude in zip (data_latitude, data_longitude) :
            start_time = time.time()
            try : 
                url = f'https://www.google.com/maps/search/Bulutangkis/@{latitude},{longitude},12.58z?entry=ttu' 
                print(f"Start in {url}")
                driver.get(url)
                time.sleep(2)
                
                try :
                    for i in range(1, 100):
                        # scroll down to bottom sidebar
                        scroll_to_bottom()

                        # xpath_endsidebar = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span')
                        if(check_exists_by_xpath('/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span')):
                            scroll_to_bottom(2)
                            print("End scroll down !!")
                            
                            break
                
                except Exception as e :
                    print(f"Ada error ni di request {url} -> {e}")
                    
                print(f'url {url} sudah selesai load')
                
                try :
                    #start crawling data !
                    scrape_data()
                except Exception as e :
                    print("Terdapat error dibagian Crawling data(Function scrape_data)")
                
            except TimeoutError :
                print(f'Teeett habis waktu jadi error di {url}')
                
            end_time = time.time()
            
            execution_time = (end_time - start_time)/60
            print(f"Waktu yang dibutuhkan {execution_time:.2f} menit")

        print("Data sudah selesai di scrapping, horee !!!")
        
    except Exception as e :
        print('error nih haduhhh')

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#


In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

class WebScraper:
    def __init__(self, driver):
        self.driver = driver
        self.all_data = []

    def check_exists_by_xpath(self, xpath):
        try:
            self.driver.find_element(By.XPATH, xpath)
        except:
            return False
        return True

    def scroll_to_bottom(self, iteration=None):
        xpath_sidebar = self.driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
        if iteration is None:
            self.driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
            time.sleep(2)
        else:
            for _ in range(1, iteration):
                self.driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
                time.sleep(2)

    def scrape_data(self):
        xpath_urls = [
            '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[1]/div[2]',
            '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[3]/div/span[2]/span/span[1]',
            '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span',
            '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[2]/span[2]',
            '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[2]/span[2]/span[2]'
        ]
        xpath_href = ['/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a']

        for i in range(3, 12, 2):
            data = []
            for xpath in xpath_urls:
                xpath = xpath.format(i)
                try:
                    WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
                    element = self.driver.find_element(By.XPATH, xpath)
                    information = element.text
                except NoSuchElementException:
                    information = ''
                data.append(information)

            for xpath_xy in xpath_href:
                xpath_xy = xpath_xy.format(i)
                try:
                    elements = self.driver.find_elements(By.XPATH, xpath_xy)
                    information_href = ''
                    for element_xy in elements:
                        information_href = element_xy.get_attribute('href')
                except NoSuchElementException:
                    information_href = ''
                data.append(information_href)

            self.all_data.append(data)

        try:
            df = pd.DataFrame(self.all_data, columns=['poi_name', 'ratings', 'tags', 'address', 'telp', 'url'])
            df.to_csv('scraped_data_testing4.csv', sep=';', index=False)
        except Exception as e:
            print(f"Error bagian saving data to csv {e}")

        for info in self.all_data:
            print(info)

    def url_hit(self, data_latitude, data_longitude):
        try:
            for latitude, longitude in zip(data_latitude, data_longitude):
                start_time = time.time()
                try:
                    url = f'https://www.google.com/maps/search/Bulutangkis/@{latitude},{longitude},12.58z?entry=ttu'
                    print(f"Start in {url}")
                    self.driver.get(url)
                    time.sleep(2)

                    try:
                        for _ in range(1, 100):
                            self.scroll_to_bottom()
                            if self.check_exists_by_xpath('/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span'):
                                self.scroll_to_bottom(2)
                                print("End scroll down !!")
                                break
                    except Exception as e:
                        print(f"Ada error ni di request {url} -> {e}")

                    print(f'url {url} sudah selesai load')

                    try:
                        self.scrape_data()
                    except Exception as e:
                        print("Terdapat error dibagian Crawling data(Function scrape_data)")

                except TimeoutError:
                    print(f'Teeett habis waktu jadi error di {url}')

                end_time = time.time()

                execution_time = (end_time - start_time) / 60
                print(f"Waktu yang dibutuhkan {execution_time:.2f} menit")

            print("Data sudah selesai di scrapping, horee !!!")

        except Exception as e:
            print('error nih haduhhh')


In [7]:
from selenium import webdriver

# Inisialisasi driver Selenium (pastikan Anda telah mengatur driver sesuai dengan browser yang ingin Anda gunakan)
driver = webdriver.Chrome()

# Inisialisasi objek WebScraper dengan driver yang telah dibuat
web_scraper = WebScraper(driver)

# Panggil metode url_hit untuk memulai proses scraping
web_scraper.url_hit(data_lat, data_long)

# Setelah selesai, Anda dapat menutup driver
driver.quit()


Start in https://www.google.com/maps/search/Bulutangkis/@-6.2725674,106.810043,12.58z?entry=ttu
End scroll down !!
url https://www.google.com/maps/search/Bulutangkis/@-6.2725674,106.810043,12.58z?entry=ttu sudah selesai load
Terdapat error dibagian Crawling data(Function scrape_data)
Waktu yang dibutuhkan 0.69 menit
Start in https://www.google.com/maps/search/Bulutangkis/@-6.59349435,106.7993574,12.58z?entry=ttu
End scroll down !!
url https://www.google.com/maps/search/Bulutangkis/@-6.59349435,106.7993574,12.58z?entry=ttu sudah selesai load
['GOR Badminton Mega Kuningan', '4,5', 'Gelanggang bulu tangkis', '8, Jl. Karet Pedurenan No.62B, RT.12/RW.4', '0822-1052-2266', 'https://www.google.com/maps/place/GOR+Badminton+Mega+Kuningan/data=!4m7!3m6!1s0x2e69f3f98411b303:0x907a3bf64017965f!8m2!3d-6.2219791!4d106.827072!16s%2Fg%2F11cs36d5mz!19sChIJA7MRhPnzaS4RX5YXQPY7epA?authuser=0&hl=id&rclk=1']
['Hall Badminton / Bulu Tangkis Santa E-Sport', '4,5', 'Lapangan Bulutangkis', 'Sebelah Golden Mans

#### Version 1.1

In [10]:

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except:
        return False
    return True


def scroll_to_bottom(iteration=None):
    xpath_sidebar = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    if(iteration == None):
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
        time.sleep(2)
    else:
        for i in range(1, iteration):
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
            time.sleep(2)


def scrape_data(all_data):
    # Daftar XPath untuk setiap informasi
    xpath_urls = [
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[1]/div[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[3]/div/span[2]/span/span[1]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[2]/span[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[2]/span[2]/span[2]'
    ]
    xpath_href = ['/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a']
    #xpath_href = ['//a[@class="hfpxzc"]']
    
    # List untuk menyimpan semua data
    #all_data = []

    # Loop untuk mengekstrak informasi dari setiap elemen
    for i in range(3, 12, 2):  # misalnya, untuk nomor 3, 5, 7, 9, 11
        data = []
        for xpath in xpath_urls:
            xpath = xpath.format(i)
            #print(xpath)
            try :
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, xpath)))
                element = driver.find_element(By.XPATH, xpath)
                information = element.text
                data.append(information)
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                data.append('')
                
        for xpath_xy in xpath_href:
            xpath_xy = xpath_xy.format(i)
            try:
                elements = driver.find_elements(By.XPATH, xpath_xy)
                for element_xy in elements:
                    information_href = element_xy.get_attribute('href')
                    #information_data = element.get_attribute('aria-label')
                    data.append(information_href)
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                data.append(('',''))
        
        all_data.append(data)
        
    # Cetak semua data yang telah diambil
    #for info in all_data:
        #print(info)
    return all_data

def save_to_csv(filename, data):
    # Tentukan header untuk file CSV
    header = ['poi_name', 'ratings', 'tags', 'address', 'telp', 'url']
    
    # Tulis data ke dalam file CSV
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')
        
        # Tulis header
        writer.writerow(header)
        
        # Tulis setiap baris data
        writer.writerows(data)

def url_hit(data_latitude, data_longitude) :
    all_data = []
    
    try : 
        for latitude, longitude in zip (data_latitude, data_longitude) :
            start_time = time.time()
            try : 
                url = f'https://www.google.com/maps/search/Bulutangkis/@{latitude},{longitude},12.58z?entry=ttu' 
                print(f"Start in {url}")
                driver.get(url)
                time.sleep(2)
                
                try :
                    for i in range(1, 100):
                        # scroll down to bottom sidebar
                        scroll_to_bottom()

                        # xpath_endsidebar = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span')
                        if(check_exists_by_xpath('/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span')):
                            scroll_to_bottom(2)
                            print("End scroll down !!")
                            
                            break
                
                except Exception as e :
                    print(f"Ada error ni di request {url} -> {e}")
                    
                print(f'url {url} sudah selesai load')
                
                try :
                    #start crawling data !
                    all_data = scrape_data(all_data)
                except Exception as e :
                    print("Terdapat error dibagian Crawling data(Function scrape_data)")
                
            except TimeoutError :
                print(f'Teeett habis waktu jadi error di {url}')
                
            end_time = time.time()
            
            execution_time = (end_time - start_time)/60
            print(f"Waktu yang dibutuhkan {execution_time:.2f} menit")

        try :    
            save_to_csv('scraped_data_testing3.csv', all_data)
        except Exception as e :
            print(f"Error bagian saving data to csv {e}")
        
        print("Data sudah selesai di scrapping, horee !!!")
        
    except Exception as e :
        print('error nih haduhhh')

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#


### Extract data poi - Testing Crawling
>> Gambaran Scriptnya 


In [18]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 

# Inisialisasi WebDriver Chrome
driver = webdriver.Chrome()

def scrape_data():
    # Daftar XPath untuk setiap informasi
    xpath_urls = [
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[1]/div[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[3]/div/span[2]/span/span[1]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[2]/span[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[2]/span[2]/span[2]'
    ]
    xpath_href = ['/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a']
    #xpath_href = ['//a[@class="hfpxzc"]']
    
    # List untuk menyimpan semua data
    all_data = []

    # Loop untuk mengekstrak informasi dari setiap elemen
    for i in range(3, 12, 2):  # misalnya, untuk nomor 3, 5, 7, 9, 11
        for xpath in xpath_urls:
            xpath = xpath.format(i)
            #print(xpath)
            try :
                element = driver.find_element(By.XPATH, xpath)
                information = element.text
                all_data.append((information))
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                all_data.append('')
                
        for xpath_xy in xpath_href:
            xpath_xy = xpath_xy.format(i)
            try:
                elements = driver.find_elements(By.XPATH, xpath_xy)
                for element_xy in elements:
                    information_href = element_xy.get_attribute('href')
                    #information_data = element.get_attribute('aria-label')
                    all_data.append((information_href))
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                all_data.append(('',''))

    # Cetak semua data yang telah diambil
    for info in all_data:
        print(info)

# URL target
url = "https://www.google.com/maps/search/bulutangkis/@-6.1712823,106.783929,14z?authuser=0&entry=ttu"
driver.get(url)

# Panggil fungsi scrape_data untuk mengeksekusi scraping
scrape_data()

# Tutup WebDriver setelah selesai
driver.quit()


Lapangan Bulutangkis Sampoerna
4,4
Lapangan Bulutangkis
Jl. Arwana I No.7, RT.13/RW.13
0856-9194-8530
https://www.google.com/maps/place/Lapangan+Bulutangkis+Sampoerna/data=!4m7!3m6!1s0x2e69f61f59751d29:0x71b668a708ae14b0!8m2!3d-6.1408544!4d106.7945255!16s%2Fg%2F11c0py6hxq!19sChIJKR11WR_2aS4RsBSuCKdotnE?authuser=0&hl=id&rclk=1

GOR Badminton PB. Samudra
4,4
Lapangan Bulutangkis
3, Jl. Angke Jaya 7 No.9, RT.2/RW.7
0818-0878-2252
https://www.google.com/maps/place/GOR+Badminton+PB.+Samudra/data=!4m7!3m6!1s0x2e69f6186b2e5c6b:0x5ed52ab301c3521b!8m2!3d-6.1456078!4d106.7955793!16s%2Fg%2F11c72xfvcb!19sChIJa1wuaxj2aS4RG1LDAbMq1V4?authuser=0&hl=id&rclk=1

Lapangan Badminton Bang Jay
4,5
Lapangan Bulutangkis
Jl. Kerajinan No.19, RT.15/RW.8
0878-8124-4816
https://www.google.com/maps/place/Lapangan+Badminton+Bang+Jay/data=!4m7!3m6!1s0x2e69f60a5ba722d3:0x55d03adaf816ad38!8m2!3d-6.1533629!4d106.8157432!16s%2Fg%2F11c1tz36xh!19sChIJ0yKnWwr2aS4ROK0W-No60FU?authuser=0&hl=id&rclk=1

Lapangan Bulu Tangkis 

In [23]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 

# Inisialisasi WebDriver Chrome
driver = webdriver.Chrome()

def scrape_data():
    # Daftar XPath untuk setiap informasi
    xpath_urls = [
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[1]/div[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[3]/div/span[2]/span/span[1]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[2]/span[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[2]/span[2]/span[2]'
    ]
    xpath_href = ['/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a']
    #xpath_href = ['//a[@class="hfpxzc"]']
    
    # List untuk menyimpan semua data
    all_data = []

    # Loop untuk mengekstrak informasi dari setiap elemen
    for i in range(3, 12, 2):  # misalnya, untuk nomor 3, 5, 7, 9, 11
        for xpath in xpath_urls:
            xpath = xpath.format(i)
            #print(xpath)
            try :
                element = driver.find_element(By.XPATH, xpath)
                information = element.text
                all_data.append((information))
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                all_data.append('')
                
        for xpath_xy in xpath_href:
            xpath_xy = xpath_xy.format(i)
            try:
                elements = driver.find_elements(By.XPATH, xpath_xy)
                for element_xy in elements:
                    information_href = element_xy.get_attribute('href')
                    #information_data = element.get_attribute('aria-label')
                    all_data.append((information_href))
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                all_data.append(('',''))

    # Cetak semua data yang telah diambil
    for info in all_data:
        print(info)

# URL target
for data_lintang, data_bujur in zip (data_lat, data_long) :
    url = f"https://www.google.com/maps/search/bulutangkis/@{data_lintang},{data_bujur},14z?authuser=0&entry=ttu"
    driver.get(url)

    # Panggil fungsi scrape_data untuk mengeksekusi scraping
    scrape_data()

    # Tutup WebDriver setelah selesai
driver.quit()


Lapangan Bulutangkis PB. CITRA DARMA 62
4,3
Lapangan Bulutangkis
Jl. Damai IV No.18C 6, RT.6/RW.2
0878-3669-3636
https://www.google.com/maps/place/Lapangan+Bulutangkis+PB.+CITRA+DARMA+62/data=!4m7!3m6!1s0x2e69f168124d3af3:0x75be6977271988a6!8m2!3d-6.260721!4d106.8024677!16s%2Fg%2F11h6qhh71r!19sChIJ8zpNEmjxaS4RpogZJ3dpvnU?authuser=0&hl=id&rclk=1

Lapangan Bulu Tangkis Angsana Ujung
4,0
Lapangan Bulutangkis
Jl. Angsana 1 Blok, Annur Blok Annur No.14, RT.1/RW.6

https://www.google.com/maps/place/Lapangan+Bulu+Tangkis+Angsana+Ujung/data=!4m7!3m6!1s0x2e69f24355fe7879:0x87f45ab76d220192!8m2!3d-6.272041!4d106.852412!16s%2Fg%2F11c0rqrl_l!19sChIJeXj-VUPyaS4RkgEibbda9Ic?authuser=0&hl=id&rclk=1

Lapangan Bulutangkis PB. BERSAUDARA
4,5
Klub Badminton
Jl. H. Eman I No.70 1, RT.1/RW.6

https://www.google.com/maps/place/Lapangan+Bulutangkis+PB.+BERSAUDARA/data=!4m7!3m6!1s0x2e69f1a9be5f2861:0xd0294af1aae306be!8m2!3d-6.2711085!4d106.776467!16s%2Fg%2F11f72vlbf0!19sChIJYShfvqnxaS4RvgbjqvFKKdA?authuser=0

In [30]:
# Inisialisasi WebDriver Chrome
driver = webdriver.Chrome()

def scrape_data():
    # Daftar XPath untuk setiap informasi
    xpath_urls = [
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[1]/div[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[3]/div/span[2]/span/span[1]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[1]/span',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[1]/span[2]/span[2]',
        '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/div[2]/div[4]/div[1]/div/div/div[2]/div[4]/div[2]/span[2]/span[2]'
    ]
    xpath_href = ['/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a']

    # List untuk menyimpan semua data
    all_data = []

    # Loop untuk mengekstrak informasi dari setiap elemen
    for i in range(3, 12, 2):  # misalnya, untuk nomor 3, 5, 7, 9, 11
        data = []
        for xpath in xpath_urls:
            xpath = xpath.format(i)
            try:
                element = driver.find_element(By.XPATH, xpath)
                information = element.text
                data.append(information)
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                data.append('')

        for xpath_xy in xpath_href:
            xpath_xy = xpath_xy.format(i)
            try:
                elements = driver.find_elements(By.XPATH, xpath_xy)
                for element_xy in elements:
                    information_href = element_xy.get_attribute('href')
                    data.append(information_href)
            except NoSuchElementException:
                # Jika elemen tidak ditemukan, tambahkan string kosong ke dalam list
                data.append('')

        all_data.append(data)

    # Buat DataFrame dari list all_data
    df = pd.DataFrame(all_data, columns=['poi_name', 'ratings', 'tags', 'address', 'telp', 'url'])

    # Simpan DataFrame ke dalam file CSV
    df.to_csv('scraped_data_testing1.csv', sep=';', index=False)
    
# URL target
url = "https://www.google.com/maps/search/bulutangkis/@-6.1712823,106.783929,14z?authuser=0&entry=ttu"
driver.get(url)

# Panggil fungsi scrape_data untuk mengeksekusi scraping
scrape_data()

# Tutup WebDriver setelah selesai
driver.quit()

### Start Crawling

In [21]:
driver = webdriver.Chrome()
url_hit(data_lat, data_long)
time.sleep(2)
driver.quit()

Start in https://www.google.com/maps/search/Bulutangkis/@-6.2725674,106.810043,12.58z?entry=ttu
End scroll down !!
url https://www.google.com/maps/search/Bulutangkis/@-6.2725674,106.810043,12.58z?entry=ttu sudah selesai load
Terdapat error dibagian Crawling data(Function scrape_data)
Waktu yang dibutuhkan 3.37 menit
Start in https://www.google.com/maps/search/Bulutangkis/@-6.59349435,106.7993574,12.58z?entry=ttu
url https://www.google.com/maps/search/Bulutangkis/@-6.59349435,106.7993574,12.58z?entry=ttu sudah selesai load
['Bulutangkis Melia Sawojajar', '4,5', 'Lapangan Bulutangkis', 'Jl. Sawojajar No.5, RT.01/RW.04', '(0251) 8326699', 'https://www.google.com/maps/place/Bulutangkis+Melia+Sawojajar/data=!4m7!3m6!1s0x2e69c5b55943b253:0x4910d5e7b11a2748!8m2!3d-6.5883242!4d106.7940623!16s%2Fg%2F11g8f_gf4z!19sChIJU7JDWbXFaS4RSCcasefVEEk?authuser=0&hl=id&rclk=1']
['GOR Bulutangkis KAYPI', '4,5', 'Klub Badminton', 'Jl. Cilubang Nagrak No.RT 003/004, RT.01/RW.04', '0816-1191-627', 'https://www